In [37]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline
plt.rcParams['figure.figsize'] = (12.0, 9.0) # set default size of plots

In [38]:
###  Modelo para Regressão Com backpropagation

def layer_sizes2(X, Y, n_h=4):
    """
    Argumentos:
    X -- shape do input (quantidade de features, quantidade de exemplos)
    Y -- shape do target (1, quantidade de exemplos)
    """
    n_x = X.shape[0]
    n_y = Y.shape[0]
    return (n_x, n_h, n_y)

def initialize_parameters2(n_x, n_h, n_y):
    """
    Argument:
    n_x -- tamanho da camada de entrada
    n_h -- tamanho da camada escondida
    n_y -- tamanho da camada de saída
    
    Retorna:
    params -- dicionário com os parâmetros (pesos) iniciais do modelo:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    

    W1 = np.random.uniform(low = -1/np.sqrt(n_h), high = 1/np.sqrt(n_h), size = (n_h, n_x))
    #W1 = np.random.uniform(low = -1/np.sqrt(n_h), high = 1/np.sqrt(n_h), size = (n_h, n_x)) * 0.01
    #b1 = np.zeros((n_h, 1))
    b1 = np.random.uniform(low = -1/np.sqrt(n_h), high = 1/np.sqrt(n_h),size = (n_h, 1))
    W2 = np.random.uniform(low = -1/np.sqrt(n_y), high = 1/np.sqrt(n_y),size = (n_y, n_h))
    #b2 = np.zeros((n_y, 1))
    b2 = np.random.uniform(low = -1/np.sqrt(n_y), high = 1/np.sqrt(n_y),size = (n_y, 1))
    
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def forward_propagation2(X, parameters):
    """
    Argumentos:
    X -- input de tamanho (n_x, m)
    parametros -- python dicionário contendo os parâmetros (saída da função de inicialização dos parametros)
    
    Retorna:
    A2 -- A saída da função sigmoidal ou tangente hiberbólica ou relu
    cache -- dicionário contendo "Z1", "A1", "Z2" e "A2"
    """

    # Recupere cada parâmetro do dicionário parameters
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Implementando a Forward Propagation para calcular A1 tanh e A2 linear
    Z1 = np.dot(W1,X) + b1
    #A1 = np.tanh(Z1)
    A1 = Z1 # linear
    Z2 = np.dot(W2, A1) + b2
    A2 = 1.7159*np.tanh(2/3 * Z2)
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache


def compute_cost2(A2, Y, parameters):
    """
    Computa o custo dado os argumentos
    
    Arguments:
    A2 -- Saída linear da segunda ativação de shape (1, qtd de exemplos)
    Y -- Valor verdadeiro do rótulo de shape (1, qtd de exemplos)
    parameters -- dicionário contendo os parâmetros W1, b1, W2 and b2
    
    Retorna:
    cost
    
    """
    
    m = Y.shape[1] # quantidade de exemplos

    # Computa o custo (cost)
    err = A2 - Y
    cost = 1/m * np.sum(err**2)
    
    cost = float(np.squeeze(cost))  # garanta que o custo tem a dimensão esperada
    assert(isinstance(cost, float))
    
    return cost

def backward_propagation2(parameters, cache, X, Y):
    """
    Implementa a retropropagação 
    
    Argumentos:
    parameters -- dicionário contendo os parâmetros
    cache -- dicionário contendo "Z1", "A1", "Z2" and "A2".
    X -- input de shape (qtd de features, qtd de examplos)
    Y -- valor verdadeiro do rótulo de shape (1, qtd de examplos)
    
    Retorna:
    grads -- dicionário contendo os gradientes em relação aos diferentes parâmetros
    """
    m = X.shape[1]
    
    # Primeiro, recuperamos W1 e W2 do dicinário "parameters"
    W1 = parameters['W1']
    W2 = parameters['W2']
        
    # Recuperamos também A1 e A2 do dicionário "cache".
    A1 = cache['A1']
    A2 = cache['A2']
    Z2 = cache['Z2']
    
    # Retropropagação: calcula-se dW1, db1, dW2, db2.
    dZ2 = (A2 - Y)* (2/3/1.7159 - np.tanh(2/3*Z2)**2)
    #dZ2 = (A2 - Y)*(1.14393 - (A2**2)/1.5)
    dW2 = 1/m * np.dot(dZ2,A1.T)
    db2 = 1/m * np.sum(dZ2, axis = 1, keepdims=True)
    #dZ1 = np.dot(W2.T, dZ2) * (1-np.power(A1, 2))
    #dZ1 = np.dot(W2.T, dZ2) * A1
    dZ1 = np.dot(W2.T, dZ2)
    dW1 = 1/m * np.dot(dZ1, X.T)
    db1 = 1/m * np.sum(dZ1, axis = 1, keepdims=True)
    

    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

def update_parameters2(parameters, grads, learning_rate = 1.2):
    """
    Atualiza os parâmetros utilizando o gradient descendente 
    
    Argumentos:
    parameters -- dicionário contendo os parâmetros
    grads -- dicionário contendo os gradientes
    
    Retorna:
    parameters -- dicionário contendo os parâmetros atualizados
    """
    # Recupera-se cada parâmetro do dicionário "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Recupera-se cada gradiente do dicionário "grads"
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    
    # Regra de atualização para cada parâmetro
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

def nn_model2(X, Y, n_h, num_iteracoes, print_cost=False):
    """
    Argumentos:
    X -- dataset de shape (2, qtd de examplos)
    Y -- labels de shape (1, qtd de examplos)
    n_h -- tamanho da camada escondida
    num_iteracoes
    print_cost -- se True, mostra o custo a cada 1000 iterações
    
    Retorna:
    parameters -- parâmetros aprendidos pelo modelo. Eles podem ser utilizados para fazer previsões (predict).
    """
    
    n_x = layer_sizes2(X, Y)[0]
    n_y = layer_sizes2(X, Y)[2]
    
    # Inicialização dos parâmetros
    parameters = initialize_parameters2(n_x, n_h, n_y)

    # Gradiente descendente (loop)
    for i in range(0, num_iteracoes):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation2(X, parameters)
        
        # Função de custo. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost2(A2, Y, parameters)
 
        # Retropropagação (Backpropagation). Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backward_propagation2(parameters, cache, X, Y)
 
        # Atualização dos parâmetros pelo gradiente descendente. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = update_parameters2(parameters, grads, learning_rate=1.2)
        
        # Print o custo (cost) a cada 1000 iterações
        if print_cost and i % 1000 == 0:
            print ("Custo após iteração %i: %f" %(i, cost))

    return parameters

def predict2(parameters, X):
    """
    Utiliza os parâmetros aprendidos para prever o valor da saída para cada exemplo X 
    
    Argumentos:
    parameters -- dicionário contendo os parâmetros
    X -- input de tamanho (n_x, m)
    
    Retorna
    predictions -- vetor de valores previstos do modelo treinado
    """
    
    A2, cache = forward_propagation2(X, parameters)
    predictions = A2
    
    return predictions

In [39]:
#### Resilient Backpropagation RProp ####

def rprop_update(parameter, grad_old, grad_new, step_size,learning_rate_max, learning_rate_min):
    
    # dimensões
    n_i = parameter.shape[0]
    n_j = parameter.shape[1]

    # transformando em vetor
    parameter.reshape(-1, 1)
    grad_old.reshape(-1, 1)
    grad_new.reshape(-1, 1)
    step_size.reshape(-1, 1)

    diffs = np.multiply(grad_old, grad_new)
    pos_indexes = np.where(diffs > 0)
    neg_indexes = np.where(diffs < 0)
    zero_indexes = np.where(diffs == 0)

    # positive
    if np.any(pos_indexes):
        # calcular o peso do step size
        step_size[pos_indexes] = np.minimum(step_size[pos_indexes] * learning_rate_max, step_size.max())       
        # calcular a direção do peso do step size
        grad_new[pos_indexes] = np.multiply(-np.sign(grad_new[pos_indexes]), step_size[pos_indexes])

        # calcular o novo peso do parâmetro
        parameter[pos_indexes] += grad_new[pos_indexes]


    if np.any(neg_indexes):
        # calcular o peso do step size
        step_size[neg_indexes] = np.maximum(step_size[neg_indexes] * learning_rate_min, step_size.min())
            
        # calcular a direção do peso 
        grad_new[neg_indexes] = 0 

    if np.any(zero_indexes):
        # calcular o peso do step size
        grad_new[zero_indexes] = np.multiply(-np.sign(grad_new[zero_indexes]), step_size[zero_indexes])
        parameter[zero_indexes] += grad_new[zero_indexes]
    

    parameter.reshape(n_i, n_j)
    grad_old.reshape(n_i, n_j)
    grad_new.reshape(n_i, n_j)
    step_size.reshape(n_i, n_j)

    return parameter, grad_new, step_size
    
    # for i in range(n_i):

    #     for j in range(n_j):

    #         if grad_old[i, j] * grad_new[i, j] > 0:
    #             step_size[i, j] = min(step_size[i,j] * learning_rate_max, step_size.max())
    #             grad_new[i, j] = np.sign(grad_new[i, j]) * step_size[i, j]
    #             #grad_new[i, j] = - np.sign(grad_new[i, j]) * step_size[i, j]
    #             #parameter[i, j] = parameter[i, j] + grad_new[i, j]
            
    #         elif grad_old[i, j] * grad_new[i, j] < 0:
    #             step_size[i, j] = max(step_size[i,j] * learning_rate_min, step_size.min())
    #             grad_new[i, j] = 0
            
    #         else:
    #             grad_new[i, j] = np.sign(grad_new[i, j]) * step_size[i, j] 
    #             #grad_new[i, j] = - np.sign(grad_new[i, j]) * step_size[i, j]
    #             #parameter[i, j] = parameter[i, j] + grad_new[i, j]

    # parameter = parameter - grad_new

    # return parameter, grad_new, step_size
    




def update_parameters_rprop2(parameters, grads_old, grads_new, step_size,learning_rate_max = 1.2, learning_rate_min = 0.5):
    """
    Atualiza os parâmetros utilizando o Resilient backpropagation
    
    Argumentos:
    parameters -- dicionário contendo os parâmetros
    grads -- dicionário contendo os gradientes
    
    Retorna:
    parameters -- dicionário contendo os parâmetros atualizados
    """
    # Recupera-se cada parâmetro do dicionário "parameters"
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    
    # Recupera-se cada gradiente do dicionário "grads_old" e "grads_new"
    dW1_old = grads_old['dW1']
    db1_old = grads_old['db1']
    dW2_old = grads_old['dW2']
    db2_old = grads_old['db2']

    dW1_new = grads_new['dW1']
    db1_new = grads_new['db1']
    dW2_new = grads_new['dW2']
    db2_new = grads_new['db2']

    W1_step = step_size['W1']
    b1_step = step_size['b1']
    W2_step = step_size['W2']
    b2_step = step_size['b2']

    W2, dW2_new, W2_step = rprop_update(W2,dW2_old, dW2_new, W2_step, learning_rate_max, learning_rate_min)
    b2, db2_new, b2_step = rprop_update(b2,db2_old, db2_new, b2_step, learning_rate_max, learning_rate_min)
    W1, dW1_new, W1_step = rprop_update(W1, dW1_old, dW1_new, W1_step, learning_rate_max, learning_rate_min)
    b1, db1_new, b1_step = rprop_update(b1,db1_old, db1_new, b1_step, learning_rate_max, learning_rate_min) 

    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    grads = {"dW1": dW1_new,
             "db1": db1_new,
             "dW2": dW2_new,
             "db2": db2_new}

    step_size = {"W1": W1_step,
                 "b1": b1_step,
                 "W2": W2_step,
                 "b2": b2_step}

    return parameters, grads, step_size

def nn_model_rprop(X, Y, n_h, num_iteracoes, print_cost=False):
    """
    Argumentos:
    X -- dataset de shape (2, qtd de examplos)
    Y -- labels de shape (1, qtd de examplos)
    n_h -- tamanho da camada escondida
    num_iteracoes -- quantidade de iterações do gradiente descendente
    print_cost -- se True, mostra o custo a cada 1000 iterações
    
    Retorna:
    parameters -- parâmetros aprendidos pelo modelo. Eles podem ser utilizados para fazer previsões (predict).
    """
    
    n_x = layer_sizes2(X, Y)[0]
    n_y = layer_sizes2(X, Y)[2]
    
    # Inicialização dos parâmetros
    parameters = initialize_parameters2(n_x, n_h, n_y)
    
    # Gradiente descendente (loop)
    grads_old = {"dW1": np.zeros((n_h, n_x)),
                 "db1": np.zeros((n_h, 1)),
                 "dW2": np.zeros((n_y, n_h)),
                 "db2": np.zeros((n_y, 1))}

    step_size = {"W1": np.random.rand(n_h, n_x),
                "b1": np.random.rand(n_h, 1),
                "W2": np.random.rand(n_y, n_h),
                "b2": np.random.rand(n_y, 1)}

    for i in range(0, num_iteracoes):
         
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation2(X, parameters)
        
        # Função de custo. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = compute_cost2(A2, Y, parameters)
 
        # Retropropagação (Backpropagation). Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads_new = backward_propagation2(parameters, cache, X, Y)
 
        # Atualização dos parâmetros pelo gradiente descendente. Inputs: "parameters, grads". Outputs: "parameters".
        parameters, grads_old, step_size = update_parameters_rprop2(parameters, grads_old, grads_new, step_size, learning_rate_max=1.2, learning_rate_min=0.5)

        # Print o custo (cost) a cada 1000 iterações
        if print_cost and i % 1000 == 0:
            print ("Custo após iteração %i: %f" %(i, cost))

    return parameters


In [40]:
### Estruturando os dados de dicionário para numpy array e de numpy array para dicionário
def parameter_dim_tot(parameter):
    '''
    Argumentos:
    parameter - array de parâmetros

    Retorna:
    dim_tot - dimensão total dos parâmetris 
    '''
    dim_tot = np.array(parameter.shape).prod()

    return dim_tot

def parameter_reshape_coluna(parameter):
    '''
    Argumentos:
    parameter - array de parâmetros

    Retorna:
    parameter_reshaped - array coluna dos parâmetros
    '''
    param_dim_tot = parameter_dim_tot(parameter)
    parameter_reshaped = parameter.reshape(1, param_dim_tot)

    return parameter_reshaped

def parameters_stack(parameters):
    '''
    Argumentos: 
    parameters - lista com os parâmetros em array 

    Retorna:
    parametros_stack - array coluna com parâmetros empilhados
    '''
    params_list = []
    param_temp = 0
    
    for param in parameters:
        param_temp = parameter_reshape_coluna(param)
        params_list.append(param_temp)
    
    params_stack = np.concatenate(tuple(params_list), axis = 1)

    return params_stack

# Unstack os parâmetros com base na dimensão dos atributos (matrizes de pesos)
def parameters_unstack(parameters_stack, atributos_dim):
    '''
    Argumentos:
    parameters_stack - array dos parâmetros no formato empilhado por colunas para trabalhar no PSO
    atributos_dim - lista com dimensão total dos atributos 

    Retorna:
    params - lista com parâmetros no formato de lista
    '''
    params = []
    i = atributos_dim[0]
    params.append(parameters_stack[:, :i])

    for dim in atributos_dim[1:]:
        params.append(parameters_stack[:, i:i+dim])
        i += dim

    return params

# Reshape para o formato do dicionário (parameters)
def parameters_reshape_dictionary(parameters_dict, parameters_unstacked):
    '''
    Argumentos:
    parameters_dict - dicionário com parâmetros
    parameters_unstacked - parâmetros no formato array 

    retorna:
    parameters_reshaped - lista com os parâmetros formatados para o dicionário 'parameters'
    '''
    w1_shape = parameters_dict['W1'].shape
    b1_shape = parameters_dict['b1'].shape
    w2_shape = parameters_dict['W2'].shape
    b2_shape = parameters_dict['b2'].shape

    parameters_reshaped = parameters_dict.copy()

    w1_reshaped = parameters_unstacked[0].reshape(w1_shape)
    b1_reshaped = parameters_unstacked[1].reshape(b1_shape)
    w2_reshaped = parameters_unstacked[2].reshape(w2_shape)
    b2_reshaped = parameters_unstacked[3].reshape(b2_shape)

    parameters_reshaped['W1'] = w1_reshaped
    parameters_reshaped['b1'] = b1_reshaped
    parameters_reshaped['W2'] = w2_reshaped
    parameters_reshaped['b2'] = b2_reshaped

    return parameters_reshaped

In [48]:
#### PSO para otimizar todos os parâmetros de uma só vez 
    
def PSO_todos(X, parameters, parameters_stacked, fun, Y, best_cost,qtd_particulas, atributos_dim, min_i, max_i, 
              max_epoch, w_in=0.7, w_fim = 0.2, c1=1.496, c2=1.496):
    '''
        Função do Algoritmo SWARM PSO. 
        Inputs:
        - fun_opt: Função de fitness a ser otimizada
        - qtd_particulas: Quantidade de partículas
        - atributos_dim: Dimensão do Vetor de atributos 
        - min: intervalo inferior do domínio da função  
        - max: intervalo superior do domínio da função
        - w: inércia 
        - c1: influência do pbest (termo cognitivo)
        - c2: influência do gbest (termo do aprendizado social)
    '''
    
    def weight_decay(w_in, w_fim, iter, iter_max):
        
        return w_in + w_fim * (1 - (iter/iter_max))


    atributos_dim_sum = sum(atributos_dim)

    # inicializar as partículas em posições aleatórias
    particulas = np.random.uniform(low = min_i, high = max_i, size = (qtd_particulas, atributos_dim_sum))

    # inicializar a velocidade
    velocidade = np.zeros((qtd_particulas, atributos_dim_sum))

    # inicializar o pbest em zero
    pbest = np.zeros((qtd_particulas,atributos_dim_sum))

    gbest_value = best_cost
    print('Custo gbest inicio PSO = ', gbest_value)

    gbest = 0
    #particulas[gbest,:] = parameters_stacked
    
    parameters_gbest_dict = parameters.copy()
    parameters_dict = parameters.copy()

    # Extrair a posição do gbest 
    for z in np.arange(qtd_particulas):
        parameters_temp = particulas[[z],:]
        parameters_temp_unstacked = parameters_unstack(parameters_temp, atributos_dim)
        parameters_temp_dict = parameters_reshape_dictionary(parameters_dict, parameters_temp_unstacked)
        A2 = predict2(parameters_temp_dict, X)
        new_value = fun(A2, Y, parameters_temp_dict)

        if new_value < gbest_value:
            gbest_value = new_value
            gbest = z
            parameters_gbest_dict = parameters_temp_dict

    # print(parameters_gbest_dict)
    print('gbest', gbest)
    
    for k in np.arange(max_epoch):

        # Atualização do decaimento do peso
        w = weight_decay(w_in, w_fim,k, max_epoch)                
        
    # Iterar para atualizar o pbest e gbest para cada partrícula
        for j in np.arange(qtd_particulas):
        
            # transformando as partículas no formato de dicionário
            parameters_temp = particulas[[j],:]
            parameters_temp_unstacked = parameters_unstack(parameters_temp, atributos_dim)
            parameters_temp_dict = parameters_reshape_dictionary(parameters_dict, parameters_temp_unstacked)

            parameters_pbest_temp = pbest[[j],:]
            parameters_pbest_temp_unstacked = parameters_unstack(parameters_pbest_temp, atributos_dim)
            parameters_pbest_dict = parameters_reshape_dictionary(parameters_dict, parameters_temp_unstacked)

            A2_part = predict2(parameters_temp_dict, X)
            A2_pbest = predict2(parameters_pbest_dict, X)
            
            # pbest
            if fun(A2_part, Y, parameters_temp_dict) < fun(A2_pbest, Y, parameters_pbest_dict):
                pbest[j,:] = particulas[j,:]

            # gbest
            if fun(A2_part, Y, parameters_temp_dict) < gbest_value:
                if np.abs(fun(A2_part, Y, parameters_temp_dict) - gbest_value) < 0.00001:
                    print('Parar o algoritmo pois o gbest não melhorou')
                    gbest_value = fun(A2_part, Y, parameters_temp_dict)
                    gbest = j
                    parameters_gbest_dict = parameters_temp_dict
                    break
                    
                print('\nCusto menor que Gbest na iteração ', k)
                print('Custo = ', fun(A2_part, Y, parameters_temp_dict))
                gbest = j
                gbest_value = fun(A2_part, Y, parameters_temp_dict)
                parameters_gbest_dict = parameters_temp_dict
                                      
         # Iteração para atualizar as posições das partículas
        for i in np.arange(qtd_particulas):
            r1, r2 = np.random.rand(), np.random.rand()
            velocidade[i, :] = w * velocidade[i, :] + c1 * r1 * (pbest[i, :] - particulas[i, :]) + c2 * r2 * (particulas[gbest, :] - particulas[i, :])
                # Atualizar partículas
            particulas[i, :] = particulas[i, :] + velocidade[i, :]

            # lidar com limites das partículas
            for dim in np.arange(atributos_dim_sum):
                if particulas[i, dim] < min_i:
                    particulas[i, dim] = min_i
                elif particulas[i, dim] > max_i:
                    particulas[i, dim] = max_i

    return parameters_gbest_dict

def update_parameters_pso_todos(X, parameters, compute_cost2, Y, best_cost, qtd_iteracoes):
    '''
    Argumentos:
    parameters - dicionário contendo os parâmetros do modelo
    compute_cost2 - função a ser minimizada, neste caso a função de custo
    A2 - previsão feita pelo modelo
    Y - rótulo 

    Retorna:
    parameters - parâmetros atualizados a partir do PSO
    '''

    # Extrair os parâmetros do dicionário para calcular a dimensão total e para criar o array colunas
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']

    # Extrair a dimensão total 
    W1_dim = np.array(W1.shape).prod()
    b1_dim = np.array(b1.shape).prod()
    W2_dim = np.array(W2.shape).prod()
    b2_dim = np.array(b2.shape).prod()

    # lista com parâmetros
    parametros = [W1, b1, W2, b2]
    # parâmetros no formato array colunas
    parameters_stacked = parameters_stack(parametros)

    atributos_dim = [W1_dim, b1_dim, W2_dim, b2_dim]

    qtd_particulas_dim = (W1.shape[1] + 1)*W1.shape[0] + (W1.shape[0] + 1)*W2.shape[0]

    parameters_pso = PSO_todos(X = X, parameters = parameters, parameters_stacked = parameters_stacked, 
                               fun = compute_cost2, Y = Y, best_cost = best_cost, qtd_particulas = qtd_particulas_dim, 
                               atributos_dim = atributos_dim, min_i = -1, max_i = 1, max_epoch = qtd_iteracoes)
    
    A2 = predict2(parameters_pso, X)
    best_cost = compute_cost2(A2, Y, parameters_pso)
    
    print('Erro de treinamento após otimizar parâmetros = ' + str(best_cost))

    return parameters_pso

def nn_model_pso_todos(X, Y, n_h, num_iteracoes, print_cost=False):
    """
    Argumentos:
    X -- dataset de shape (2, qtd de exemplos)
    Y -- labels de shape (1, qtd de exemplos)
    n_h -- tamanho da camada escondida
    num_iteracoes
    print_cost -- se True, mostra o custo a cada 1000 iterações
    
    Retorna:
    parameters -- parâmetros aprendidos pelo pso. Eles podem ser utilizados para fazer previsões (predict).
    """
    
    n_x = layer_sizes2(X, Y)[0]
    n_y = layer_sizes2(X, Y)[2]
    
    # Inicialização dos parâmetros
    parameters = initialize_parameters2(n_x, n_h, n_y)
    
    A2, _ = forward_propagation2(X, parameters)

    best_cost = compute_cost2(A2, Y, parameters)
    
    # Atualização dos parâmetros pelo gradiente descendente. Inputs: "parameters, compute_cost2, A2, Y". Outputs: "parameters".
    parameters = update_parameters_pso_todos(X, parameters, compute_cost2, Y, best_cost, num_iteracoes)

    # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
    A2, _ = forward_propagation2(X, parameters)

    # Função de custo. Inputs: "A2, Y, parameters". Outputs: "cost".
    cost_new = compute_cost2(A2, Y, parameters)
    
    return parameters
    
    
#     for i in range(0, num_iterations):
 
#         # Atualização dos parâmetros pelo gradiente descendente. Inputs: "parameters, compute_cost2, A2, Y". Outputs: "parameters".
#         parameters = update_parameters_pso_todos(X, parameters, compute_cost2, Y, best_cost, num_iterations)

#         # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
#         A2, _ = forward_propagation2(X, parameters)

#         # Função de custo. Inputs: "A2, Y, parameters". Outputs: "cost".
#         cost_new = compute_cost2(A2, Y, parameters)

#         # Avaliar se a nova busca melhorou
#         if cost_new < best_cost:
#             parameters_best = parameters
#             best_cost = cost_new

#         # Print o custo (cost) a cada 5 iterações
#         if print_cost and i % 2 == 0:
#             print ("Custo após iteração %i: %f" %(i, cost_new))
    
#     return parameters_best, best_cost



In [42]:
### Experimento com as as séries do artigo 

## Importar funções para pre-processar os dados
from funcoes import split_sequence, divisao_dados_temporais, normalizar_serie, desnormalizar, cenarios_dinamicos, cmf, gf

## Sunspot

Série anual.

Inputs: 10

In [19]:
## Sunspot
sunspot = pd.read_csv('dados/sunspot.csv')
sunspot = sunspot['valor']
sunspot.head()

0     5.0
1    11.0
2    16.0
3    23.0
4    36.0
Name: valor, dtype: float64

In [20]:
sunspot_norm = normalizar_serie(sunspot)
X, y = split_sequence(sunspot_norm.values, 10, 1)

X_treino, y_treino, X_teste, y_teste, X_val, y_val = divisao_dados_temporais(X, y, perc_treino = 0.56, perc_val = 0.24)
# para avaliar mse
y_teste_ = desnormalizar(y_teste, sunspot)

In [ ]:
### Treinar rede neural com Rprop

##### Testar 5 neurônios na camada escondida e 10 execuções
best_model = 0
best_mse = np.inf

for exec in range(1):
    print('Execução ' + str(exec))
    parameters = nn_model_rprop(X_treino.T, y_treino.T, n_h=5, print_cost=True)
    y_pred = predict2(parameters, X_val.T)
    mse_exec = compute_cost2(y_pred,y_val.T, parameters)
    
    if mse_exec < best_mse:
        best_model = parameters 
        best_mse = mse_exec

    print('MSE validação = ' + str(mse_exec))


In [ ]:
# Desnormalizar
y_pred_teste = predict2(best_model, X_teste.T).reshape(-1,1)

# mse de teste
print("mse de teste = ",((y_pred_teste - y_teste)**2).mean())

y_pred_teste = desnormalizar(y_pred_teste, sunspot)

In [ ]:
# Plot
plt.plot(y_teste_, color = 'blue')
plt.plot(y_pred_teste, color = 'coral');

In [ ]:
### Treinar rede neural com backprop

##### Testar 4 neurônios na camada escondida e 10000 epocas com 30 execuções
best_model = 0
best_mse = np.inf

#neuronios = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
neuronios = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in neuronios:
    print('Neurônios: ', i)

    for exec in np.arange(1):
        print('Execução ' + str(exec))
        parameters = nn_model2(X_treino.T, y_treino.T, n_h = i, print_cost = True)
        y_pred = predict2(parameters, X_val.T)
        mse_exec = compute_cost2(y_pred,y_val.T, parameters)
    
        if mse_exec < best_mse:
            best_model = parameters 
            best_mse = mse_exec
            print('Melhor MSE: ', best_mse)
            qtd_neuronios = i

    
    print('MSE validação = ' + str(mse_exec))

In [ ]:
# melhor configuração
print('NN com {} neurônios na camada escondida'.format(qtd_neuronios))

In [ ]:
print(best_model)

In [ ]:
# Desnormalizar
y_pred_teste = predict2(best_model, X_teste.T).reshape(-1,1)

# mse de teste
print("mse de teste = ",((y_pred_teste - y_teste)**2).mean())

y_pred_teste = desnormalizar(y_pred_teste, sunspot)

In [ ]:
# Plot
plt.plot(y_teste_, label = 'Teste')
plt.plot(y_pred_teste, color = 'coral', label = 'NN com Backprop')
plt.legend();

In [ ]:
### Treinar rede neural com pso

##### Testar 5 neurônios na camada escondida
best_model = 0
best_mse = np.inf

neuronios = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
#neuronios = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for i in neuronios:
    print('Neurônios: ', i)

    for exec in np.arange(1):

        print('Execução ' + str(exec))
        parameters,_ = nn_model_pso_todos(X_treino.T, y_treino.T, n_h = i, print_cost = True)
    
        y_pred = predict2(parameters, X_val.T)
        mse_exec = compute_cost2(y_pred, y_val.T,parameters)
    
        if mse_exec < best_mse:
            best_model = parameters 
            
            best_mse = mse_exec
            print('Melhor MSE: ', best_mse)
            qtd_neuronios = i

        print('Neurônios' + str(i) + ' -> MSE validação = ' + str(mse_exec))

In [ ]:
print('NN com {} neurônios na camada escondida'.format(qtd_neuronios))

In [ ]:
# Desnormalizar
y_pred_teste = predict2(best_model, X_teste.T).reshape(-1,1)

# mse de teste
print("mse de teste = ",((y_pred_teste - y_teste)**2).mean())

y_pred_teste = desnormalizar(y_pred_teste, sunspot)

In [ ]:
best_model

In [ ]:
# Plot
plt.plot(y_teste_, label = 'Teste')
plt.plot(y_pred_teste, label='NN com PSO')
#plt.plot(y_teste_, color = 'blue', legend = 'Teste')
#plt.plot(y_pred_teste, color = 'coral', legend = 'NN com PSO')
plt.legend();

In [ ]:
# Só previsão
plt.plot(y_pred_teste, color = 'coral',label = 'NN com PSO')
plt.legend();

#### Cenário I - Sunspot

Testando Cenário I

* w = 60
* s = 10
* f = 50

In [43]:
# Criando cenários
def cenarios_execucoes(X, y, w, s, f, modelo, qtd_execucoes = 30):
    
    X_I = cenarios_dinamicos(X, 60, 10)
    y_I = cenarios_dinamicos(y, 60, 10)

    mse_treino = np.zeros((qtd_execucoes, len(y_I)))
    mse_teste = np.zeros((qtd_execucoes, len(y_I)))

    execucoes = np.arange(qtd_execucoes)

    for execucao in execucoes:

        print('Execução: ', execucao)
    
        # Janelamento
        for i in np.arange(len(y_I)):
    
            ## Divisão em treinamento e teste
            X_treino, y_treino, X_teste, y_teste, X_validacao, y_validacao = divisao_dados_temporais(X_I[i], y_I[i], perc_treino=.56, perc_val = .24)
    
            ### Treinar rede neural com backprop
            # setando parâmetros para comparação
            best_model = 0
            best_mse = np.inf

            # quantidade de neurônios de 2 até 25
            neuronios = np.arange(2, 26)
    
            # grid search 
            for j in neuronios:
                #print('Neurônios: ', j)
        
                # treinar NN para f iterações
                parameters = modelo(X_treino.T, y_treino.T, n_h = j, num_iteracoes = f)
        
                # predição na validação
                y_pred_val = predict2(parameters, X_validacao.T)
                mse_validacao = compute_cost2(y_pred_val, y_validacao.T, parameters)
        
                if mse_validacao < best_mse:
                    best_model = parameters 
                    best_mse = mse_validacao
                    #print('Melhor MSE: ', best_mse)
                    qtd_neuronios = j
                
            # retreinar e retestar com a melhor topologia 
            y_pred_treino = predict2(best_model, X_treino.T)
            mse_treino_temp = compute_cost2(y_pred_treino, y_treino.T, best_model)
            mse_treino[execucoes,i] = mse_treino_temp
        
            y_pred_teste = predict2(best_model, X_teste.T)
            mse_teste_temp = compute_cost2(y_pred_teste, y_teste.T, best_model)
            mse_teste[execucoes,i] = mse_teste_temp

    return mse_treino, mse_teste

In [44]:
## Criando avaliação dos resultados
def avaliacao_resultados(mse_treino, mse_teste, T):
    
    te = np.apply_along_axis(cmf, 1, mse_treino, T)
    ge = np.apply_along_axis(cmf, 1, mse_teste, T)

    # calcular a métrica fator de generalização
    gf = ge/te

    # Média e desvio padrão
    te_medio = te.mean()
    te_std = te.std()

    ge_medio = ge.mean()
    ge_std = ge.std()

    gf_medio = gf.mean()
    gf_std = gf.std()

    print('TE medio: ', te_medio)
    print('TE desvio: ', te_std)
    print('GE medio: ', ge_medio)
    print('GE desvio: ', ge_std)
    print('GF medio: ', gf_medio)
    print('GF desvio: ', gf_std)
    
    resultados = {'TE medio': te_medio,
    'TE desvio': te_std,
    'GE medio': ge_medio,
    'GE desvio':ge_std,
    'GF medio':gf_medio,
    'GF desvio':gf_std}
    
    return resultados

In [51]:
# Quantidade total de iterações para o primeiro cenário

# Seed
np.random.seed(3)

w = 60 # tamanho da janela
s = 10 # tamanho do passo
f = 50 # quantidade de iterações para a janela
T = f/s*(len(y))+f
print(T)

1445.0


In [ ]:
# backprop
print('BACKPROP')
mse_treino_1_backprop, mse_teste_1_backprop = cenarios_execucoes(X, y, w, s, f, modelo = nn_model2)
#resultados_1_backprop = avaliacao_resultados(mse_treino_1_backprop, mse_teste_1_backprop, T) 

print('PSO')
# pso
mse_treino_1_pso, mse_teste_1_pso = cenarios_execucoes(X, y, w, s, f, modelo = nn_model_pso_todos)
resultados_1_pso = avaliacao_resultados(mse_treino_1_pso, mse_teste_1_pso, T) 

BACKPROP
Execução:  0
Execução:  1
Execução:  2
Execução:  3
Execução:  4
Execução:  5
Execução:  6
Execução:  7
Execução:  8
Execução:  9
Execução:  10
Execução:  11
Execução:  12
Execução:  13
Execução:  14
Execução:  15
Execução:  16
Execução:  17
Execução:  18
Execução:  19
Execução:  20
Execução:  21
Execução:  22
Execução:  23
Execução:  24
Execução:  25
Execução:  26
Execução:  27
Execução:  28
Execução:  29
PSO
Execução:  0
Custo gbest inicio PSO =  1.101467096063771
gbest 1

Custo menor que Gbest na iteração  1
Custo =  0.0010802903633914008

Custo menor que Gbest na iteração  2
Custo =  0.0004435959829643932

Custo menor que Gbest na iteração  17
Custo =  0.0004212695703515198
Erro de treinamento após otimizar parâmetros = 0.0004212695703515198
Custo gbest inicio PSO =  0.16931949163622242
gbest 8

Custo menor que Gbest na iteração  1
Custo =  0.0007386455992493795

Custo menor que Gbest na iteração  1
Custo =  0.0004964923829972092

Custo menor que Gbest na iteração  1
Custo

#### Cenário II - Sunspot

Testando Cenário II

* w = 60
* s = 20
* f = 100

In [ ]:
w = 60 # tamanho da janela
s = 20 # tamanho do passo
f = 100 # quantidade de iterações para a janela
T = f/s*(len(y))+f
print('Quantidade total de iterações: ', T)

In [ ]:
# backprop
print('BACKPROP')
mse_treino_2_backprop, mse_teste_2_backprop = cenarios_execucoes(X, y, w, s, f, modelo = nn_model2)
resultados_2_backprop = avaliacao_resultados(mse_treino_2_backprop, mse_teste_2_backprop, T) 

print('PSO')
# pso
mse_treino_2_pso, mse_teste_2_pso = cenarios_execucoes(X, y, w, s, f, modelo = nn_model_pso_todos)
resultados_2_pso = avaliacao_resultados(mse_treino_2_pso, mse_teste_2_pso, T) 

#### Cenário III - Sunspot

Testando Cenário III

* w = 60
* s = 40
* f = 150

In [ ]:
w = 60 # tamanho da janela
s = 40 # tamanho do passo
f = 150 # quantidade de iterações para a janela
T = f/s*(len(y))+f
print('Quantidade total de iterações: ', T)

In [ ]:
# backprop
print('BACKPROP')
mse_treino_3_backprop, mse_teste_3_backprop = cenarios_execucoes(X, y, w, s, f, modelo = nn_model2)
resultados_3_backprop = avaliacao_resultados(mse_treino_3_backprop, mse_teste_3_backprop, T) 

print('PSO')
# pso
mse_treino_3_pso, mse_teste_3_pso = cenarios_execucoes(X, y, w, s, f, modelo = nn_model_pso_todos)
resultados_3_pso = avaliacao_resultados(mse_treino_3_pso, mse_teste_3_pso, T) 

#### Cenário IV - Sunspot

Testando Cenário IV

* w = 60
* s = 60
* f = 100

In [ ]:
w = 60 # tamanho da janela
s = 60 # tamanho do passo
f = 100 # quantidade de iterações para a janela
T = f/s*(len(y))+f
print('Quantidade total de iterações: ', T)

In [ ]:
# backprop
print('BACKPROP')
mse_treino_4_backprop, mse_teste_4_backprop = cenarios_execucoes(X, y, w, s, f, modelo = nn_model2)
resultados_4_backprop = avaliacao_resultados(mse_treino_4_backprop, mse_teste_4_backprop, T) 

print('PSO')
# pso
mse_treino_4_pso, mse_teste_4_pso = cenarios_execucoes(X, y, w, s, f, modelo = nn_model_pso_todos)
resultados_4_pso = avaliacao_resultados(mse_treino_4_pso, mse_teste_4_pso, T) 

## Airline Time Series

Série mensal
Inputs: 12

In [ ]:
## Airline Passenger
airline = pd.read_csv('dados/airline_passengers.csv')
airline = airline['valor']
airline.head()